<div style="background-color: RGB(51,165,182);" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">1 - Importation des librairies et chargement des fichiers</h2>
</div>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from datetime import datetime
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import plotly.express as px
import scipy.cluster.hierarchy as sch
from sklearn.preprocessing import PowerTransformer, StandardScaler, MinMaxScaler
from scipy.stats import kruskal
from scipy.stats import boxcox

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">1.2 - Chargements des fichiers</h3>
</div>

In [2]:
df_pop = pd.read_csv('data/Population_2000_2018.csv')
df_world = pd.read_csv('data/donnees_mondiales_2017.csv')
df_food =pd.read_csv('data/DisponibiliteAlimentaire_2017.csv')
df_internet = pd.read_csv("data/internet.csv",sep=';', encoding='ISO-8859-1')
df_iso = pd.read_csv("data/mapping.csv")

<div style="background-color: RGB(51,165,182);" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">2 - Analyse exploratoire des fichiers</h2>
</div>

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">2.1 - Analyse Pop</h3>
</div>

In [3]:
display(df_pop.head(5))
df_pop.info()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2000,2000,1000 personnes,20779.953,X,Sources internationales sûres,NaN
1,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2001,2001,1000 personnes,21606.988,X,Sources internationales sûres,NaN
2,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2002,2002,1000 personnes,22600.770,X,Sources internationales sûres,NaN
3,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2003,2003,1000 personnes,23680.871,X,Sources internationales sûres,NaN
4,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2004,2004,1000 personnes,24726.684,X,Sources internationales sûres,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4411 entries, 0 to 4410
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Code Domaine            4411 non-null   object 
 1   Domaine                 4411 non-null   object 
 2   Code zone               4411 non-null   int64  
 3   Zone                    4411 non-null   object 
 4   Code Élément            4411 non-null   int64  
 5   Élément                 4411 non-null   object 
 6   Code Produit            4411 non-null   int64  
 7   Produit                 4411 non-null   object 
 8   Code année              4411 non-null   int64  
 9   Année                   4411 non-null   int64  
 10  Unité                   4411 non-null   object 
 11  Valeur                  4411 non-null   float64
 12  Symbole                 4411 non-null   object 
 13  Description du Symbole  4411 non-null   object 
 14  Note                    258 non-null    

In [4]:
df_pop = df_pop.drop(['Code Domaine','Domaine','Code zone','Code Élément','Code Produit','Produit','Code année','Unité','Symbole','Description du Symbole','Note'], axis=1)
display(df_pop.head(5))

,Zone,Élément,Année,Valeur
0,Afghanistan,Population totale,2000,20779.953
1,Afghanistan,Population totale,2001,21606.988
2,Afghanistan,Population totale,2002,22600.770
3,Afghanistan,Population totale,2003,23680.871
4,Afghanistan,Population totale,2004,24726.684


In [5]:
df_pop_2017 = df_pop[df_pop['Année'] == 2017]
print(df_pop_2017.head())

              Zone            Élément  Année     Valeur
17     Afghanistan  Population totale   2017  36296.113
36  Afrique du Sud  Population totale   2017  57009.756
55         Albanie  Population totale   2017   2884.169
74         Algérie  Population totale   2017  41389.189
93       Allemagne  Population totale   2017  82658.409


In [6]:
df_pop_2017 = df_pop_2017.rename(columns={'Zone': 'Area'})

In [7]:
df_pivot2 = df_pop_2017.pivot(index='Area', columns=['Élément'], values='Valeur').reset_index()
df_pivot2.columns = ['_'.join(map(str, col)).strip() if isinstance(col, tuple) else col for col in df_pivot2.columns.values]
df_pivot2

,Area,Population totale
0,Afghanistan,36296.113
1,Afrique du Sud,57009.756
2,Albanie,2884.169
3,Algérie,41389.189
4,Allemagne,82658.409
...,...,...
231,Îles Salomon,636.039
232,Îles Turques-et-Caïques,37.115
233,Îles Vierges américaines,104.751
234,Îles Vierges britanniques,29.577


<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">2.2 - Analyse Food</h3>
</div>

In [8]:
df_food.head()

,Code Domaine,Domaine,Code zone,Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
0,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5511,Production,2511,Blé et produits,2017,2017,Milliers de tonnes,4281.0,S,Données standardisées
1,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5611,Importations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,2302.0,S,Données standardisées
2,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5072,Variation de stock,2511,Blé et produits,2017,2017,Milliers de tonnes,-119.0,S,Données standardisées
3,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5911,Exportations - Quantité,2511,Blé et produits,2017,2017,Milliers de tonnes,0.0,S,Données standardisées
4,FBS,Nouveaux Bilans Alimentaire,2,Afghanistan,5301,Disponibilité intérieure,2511,Blé et produits,2017,2017,Milliers de tonnes,6701.0,S,Données standardisées


In [9]:
df_food = df_food.drop(['Code Domaine','Domaine','Code zone','Code Élément','Code Produit','Code année','Unité','Symbole','Description du Symbole'], axis=1)
df_food.head()

,Zone,Élément,Produit,Année,Valeur
0,Afghanistan,Production,Blé et produits,2017,4281.0
1,Afghanistan,Importations - Quantité,Blé et produits,2017,2302.0
2,Afghanistan,Variation de stock,Blé et produits,2017,-119.0
3,Afghanistan,Exportations - Quantité,Blé et produits,2017,0.0
4,Afghanistan,Disponibilité intérieure,Blé et produits,2017,6701.0


In [10]:
df_Volailles = df_food[df_food['Produit'] == 'Viande de Volailles']

df_Volailles.head()

,Zone,Élément,Produit,Année,Valeur
651,Afghanistan,Production,Viande de Volailles,2017,28.0
652,Afghanistan,Importations - Quantité,Viande de Volailles,2017,29.0
653,Afghanistan,Variation de stock,Viande de Volailles,2017,0.0
654,Afghanistan,Disponibilité intérieure,Viande de Volailles,2017,57.0
655,Afghanistan,Pertes,Viande de Volailles,2017,2.0


In [11]:
df_Volailles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2061 entries, 651 to 176400
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Zone     2061 non-null   object 
 1   Élément  2061 non-null   object 
 2   Produit  2061 non-null   object 
 3   Année    2061 non-null   int64  
 4   Valeur   2061 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 96.6+ KB


In [12]:
df_pivot1 = df_Volailles.pivot(index='Zone', columns=['Élément', 'Produit'], values='Valeur').reset_index()
df_pivot1.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df_pivot1.columns.values]
df_pivot1

,Zone_,Production_Viande de Volailles,Importations - Quantité_Viande de Volailles,Variation de stock_Viande de Volailles,Disponibilité intérieure_Viande de Volailles,Pertes_Viande de Volailles,Résidus_Viande de Volailles,Nourriture_Viande de Volailles,Disponibilité alimentaire en quantité (kg/personne/an)_Viande de Volailles,Disponibilité alimentaire (Kcal/personne/jour)_Viande de Volailles,Disponibilité de protéines en quantité (g/personne/jour)_Viande de Volailles,Disponibilité de matière grasse en quantité (g/personne/jour)_Viande de Volailles,Exportations - Quantité_Viande de Volailles,Alimentation pour touristes_Viande de Volailles,Traitement_Viande de Volailles,Autres utilisations (non alimentaire)_Viande de Volailles,Aliments pour animaux_Viande de Volailles,Semences_Viande de Volailles
0,Afghanistan,28.0,29.0,0.0,57.0,2.0,0.0,55.0,1.53,5.0,0.54,0.33,NaN,NaN,NaN,NaN,NaN,NaN
1,Afrique du Sud,1667.0,514.0,-0.0,2118.0,83.0,0.0,2035.0,35.69,143.0,14.11,9.25,63.0,0.0,NaN,NaN,NaN,NaN
2,Albanie,13.0,38.0,4.0,47.0,NaN,0.0,47.0,16.36,85.0,6.26,6.45,0.0,NaN,NaN,NaN,NaN,NaN
3,Algérie,275.0,2.0,0.0,277.0,13.0,0.0,264.0,6.38,22.0,1.97,1.50,0.0,0.0,NaN,NaN,NaN,NaN
4,Allemagne,1514.0,842.0,-29.0,1739.0,NaN,-38.0,1609.0,19.47,71.0,7.96,4.16,646.0,NaN,167.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,Émirats arabes unis,48.0,433.0,-26.0,412.0,NaN,-0.0,412.0,43.47,147.0,14.80,9.25,94.0,NaN,NaN,NaN,NaN,NaN
168,Équateur,340.0,0.0,-1.0,341.0,17.0,0.0,324.0,19.31,83.0,6.15,6.35,0.0,0.0,NaN,NaN,NaN,NaN
169,États-Unis d'Amérique,21914.0,123.0,80.0,18266.0,NaN,-0.0,18100.0,55.68,219.0,19.93,14.83,3692.0,NaN,77.0,89.0,NaN,NaN
170,Éthiopie,14.0,1.0,0.0,14.0,1.0,0.0,14.0,0.13,0.0,0.04,0.03,NaN,0.0,NaN,NaN,NaN,NaN


In [13]:
df_pivot1.isnull().mean()

Zone_                                                                                0.000000
Production_Viande de Volailles                                                       0.023256
Importations - Quantité_Viande de Volailles                                          0.011628
Variation de stock_Viande de Volailles                                               0.017442
Disponibilité intérieure_Viande de Volailles                                         0.011628
Pertes_Viande de Volailles                                                           0.610465
Résidus_Viande de Volailles                                                          0.046512
Nourriture_Viande de Volailles                                                       0.011628
Disponibilité alimentaire en quantité (kg/personne/an)_Viande de Volailles           0.000000
Disponibilité alimentaire (Kcal/personne/jour)_Viande de Volailles                   0.000000
Disponibilité de protéines en quantité (g/personne/jour)_Via

In [14]:
colonnes_avant = len(df_pivot1.columns)
df_pivot1 = df_pivot1.dropna(axis=1, thresh=int(len(df_pivot1) * 0.7))
colonnes_apres = len(df_pivot1.columns)
print(f"Le DataFrame a été nettoyé de {colonnes_avant - colonnes_apres} colonnes.")

Le DataFrame a été nettoyé de 6 colonnes.


In [15]:
df_pivot1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 12 columns):
 #   Column                                                                             Non-Null Count  Dtype  
---  ------                                                                             --------------  -----  
 0   Zone_                                                                              172 non-null    object 
 1   Production_Viande de Volailles                                                     168 non-null    float64
 2   Importations - Quantité_Viande de Volailles                                        170 non-null    float64
 3   Variation de stock_Viande de Volailles                                             169 non-null    float64
 4   Disponibilité intérieure_Viande de Volailles                                       170 non-null    float64
 5   Résidus_Viande de Volailles                                                        164 non-null    float64

In [16]:
df_pivot1 = df_pivot1.rename(columns={'Zone_': 'Area'})

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">2.3 - Analyse WB</h3>
</div>

In [17]:
df_world

,Area,PIB par habitant,Population totale,Taux d'urbanisation,Taux de chômage,Taux d'inflation,Dette publique (% du PIB),Part des énergies renouvelables,Taux de mortalité infantile,Taux d'électrification
0,Africa Eastern and Southern,1485.753579,632746570.0,35.358901,7.123032,6.399343,NaN,61.891132,46.422853,40.199898
1,Africa Western and Central,1590.555785,431138704.0,46.039478,4.204170,1.725486,NaN,76.413081,66.304602,48.789457
2,Arab World,6068.603008,423664839.0,58.371290,11.068716,2.069329,NaN,4.375673,26.893261,90.666302
3,Caribbean small states,11576.276969,3016651.0,46.320181,10.961184,1.518207,NaN,13.463039,18.051597,95.743033
4,Central Europe and the Baltics,14225.877325,102740078.0,62.248455,5.297647,2.075936,NaN,16.467021,4.443614,99.869143
...,...,...,...,...,...,...,...,...,...,...
261,Virgin Islands (U.S.),35365.069304,107281.0,95.603000,11.955000,NaN,NaN,5.300000,NaN,100.000000
262,West Bank and Gaza,3620.360487,4454805.0,75.894000,25.677000,0.212571,NaN,10.500000,14.700000,99.900000
263,"Yemen, Rep.",893.716501,30034389.0,36.016000,18.564000,NaN,NaN,3.000000,36.800000,79.200000
264,Zambia,1495.752138,17298054.0,42.976000,8.518000,6.577312,52.280942,83.500000,44.000000,40.300000


In [18]:
df_world.isnull().mean()

Area                               0.000000
PIB par habitant                   0.030075
Population totale                  0.003759
Taux d'urbanisation                0.011278
Taux de chômage                    0.116541
Taux d'inflation                   0.139098
Dette publique (% du PIB)          0.729323
Part des énergies renouvelables    0.022556
Taux de mortalité infantile        0.082707
Taux d'électrification             0.011278
dtype: float64

In [19]:
colonnes_avant = len(df_world.columns)
colonnes_manquantes = df_world.columns[df_world.isnull().mean() > 0.3]

print(colonnes_manquantes)


df_world = df_world.dropna(axis=1, thresh=int(len(df_world) * 0.7))

colonnes_apres = len(df_world.columns)
print(f"Le DataFrame a été nettoyé de {colonnes_avant - colonnes_apres} colonnes.")

Index(['Dette publique (% du PIB)'], dtype='object')
Le DataFrame a été nettoyé de 1 colonnes.


In [20]:
df_world.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 9 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Area                             266 non-null    object 
 1   PIB par habitant                 258 non-null    float64
 2   Population totale                265 non-null    float64
 3   Taux d'urbanisation              263 non-null    float64
 4   Taux de chômage                  235 non-null    float64
 5   Taux d'inflation                 229 non-null    float64
 6   Part des énergies renouvelables  260 non-null    float64
 7   Taux de mortalité infantile      244 non-null    float64
 8   Taux d'électrification           263 non-null    float64
dtypes: float64(8), object(1)
memory usage: 18.8+ KB


<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">2.4 - Analyse internet</h3>
</div>

In [21]:
df_internet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Country Name  264 non-null    object 
 1   Country Code  266 non-null    object 
 2   2017          253 non-null    float64
dtypes: float64(1), object(2)
memory usage: 6.4+ KB


In [22]:
df_internet = df_internet.rename(columns={'Country Name': 'Area', '2017': 'Internet'})

In [23]:
df_internet['Internet'] = pd.to_numeric(df_internet['Internet'], errors='coerce')

In [24]:
df_internet.head()

,Area,Country Code,Internet
0,Aruba,ABW,97.170000
1,NaN,AFE,17.310987
2,Afghanistan,AFG,13.500000
3,NaN,AFW,27.081219
4,Angola,AGO,26.000000


<div style="background-color: RGB(51,165,182);" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">3 - Fusion</h2>
</div>

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">3.1 - Verification des pays avant de les fusionner</h3>
</div>

In [25]:
# Uniformiser les noms de pays 
df_world = df_world.copy()
df_pivot1 = df_pivot1.copy()
df_pivot2 = df_pivot2.copy()
df_internet = df_internet.copy()
df_world['Area'] = df_world['Area'].str.strip().str.lower()
df_pivot1['Area'] = df_pivot1['Area'].str.strip().str.lower()
df_pivot2['Area'] = df_pivot2['Area'].str.strip().str.lower()
df_internet['Area'] = df_internet['Area'].str.strip().str.lower()

In [26]:
# Fonction pour optimiser les types de données
def optimize_dtypes(df):
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = df[col].astype('float32')
    for col in df.select_dtypes(include=(['int64'])).columns:
        df[col] = df[col].astype('int32')
    return df

# Étape 1 : Harmoniser les datasets avec df_iso
df_internet_harmonized = pd.merge(
    df_internet[['Area', 'Internet']], df_iso[['Country', 'Alpha-3 code']],
    left_on='Area', right_on='Country', how='left'
).drop(columns=['Country']).set_index('Alpha-3 code')

df_pivot1_harmonized = pd.merge(
    df_pivot1[['Area', 'Production_Viande de Volailles']], df_iso[['PAYS', 'Alpha-3 code']],
    left_on='Area', right_on='PAYS', how='left'
).drop(columns=['PAYS']).set_index('Alpha-3 code')

df_pivot2_harmonized = pd.merge(
    df_pivot2[['Area', 'Population totale']], df_iso[['PAYS', 'Alpha-3 code']],
    left_on='Area', right_on='PAYS', how='left'
).drop(columns=['PAYS']).set_index('Alpha-3 code')

df_world_harmonized = pd.merge(
    df_world[['Area', 'PIB par habitant']], df_iso[['Country', 'Alpha-3 code']],
    left_on='Area', right_on='Country', how='left'
).drop(columns=['Country']).set_index('Alpha-3 code')

# Optimiser les types de données
df_internet_harmonized = optimize_dtypes(df_internet_harmonized)
df_pivot1_harmonized = optimize_dtypes(df_pivot1_harmonized)
df_pivot2_harmonized = optimize_dtypes(df_pivot2_harmonized)
df_world_harmonized = optimize_dtypes(df_world_harmonized)

# Étape 2 : Fusion progressive sans chunking
df_final = df_internet_harmonized.merge(
    df_pivot1_harmonized, left_index=True, right_index=True, how='outer'
).merge(
    df_pivot2_harmonized, left_index=True, right_index=True, how='outer'
).merge(
    df_world_harmonized, left_index=True, right_index=True, how='outer'
)

# Optimiser les types de données du dataset final
df_final = optimize_dtypes(df_final)

# Afficher les premières lignes du dataset final
print(df_final.head())

# Sauvegarder le fichier final
df_final.to_csv('final_dataset_cleaned.csv', index=True)


MemoryError: Unable to allocate 10.1 MiB for an array with shape (2660000, 1) and data type float32

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">3.2 - Fusion</h3>
</div>

In [262]:
df_final.head()

,Area_1,PIB par habitant,Population totale_2,Taux d'urbanisation,Taux de chômage,Taux d'inflation,Part des énergies renouvelables,Taux de mortalité infantile,Taux d'électrification,Area_translated_1,...,Disponibilité de protéines en quantité (g/personne/jour)_Viande de Volailles,Disponibilité de matière grasse en quantité (g/personne/jour)_Viande de Volailles,Exportations - Quantité_Viande de Volailles,Area_translated_2,Area_3,Population totale_3,Area_translated_3,Area_4,Internet,Area_translated_4
0,Afghanistan,526.140801,35643418.0,25.250,11.184,4.975952,19.5,51.5,97.7,afghanistan,...,0.54,0.33,NaN,afghanistan,Afghanistan,36296.113,afghanistan,Afghanistan,13.500000,afghanistan
1,Albania,4531.032207,2873457.0,59.383,13.616,1.986661,36.9,8.3,99.9,albania,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Albania,62.400000,albania
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.26,6.45,0.0,albanie,Albania,2884.169,albanie,NaN,NaN,NaN
3,Algeria,4615.868744,41136546.0,72.052,12.000,5.591116,0.1,21.0,99.5,algeria,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Algeria,47.691055,algeria
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.97,1.50,0.0,algerie,Algeria,41389.189,algerie,NaN,NaN,NaN


In [228]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169 entries, 0 to 168
Data columns (total 30 columns):
 #   Column                                                                             Non-Null Count  Dtype  
---  ------                                                                             --------------  -----  
 0   Area_1                                                                             135 non-null    object 
 1   PIB par habitant                                                                   135 non-null    float64
 2   Population totale_2                                                                135 non-null    float64
 3   Taux d'urbanisation                                                                135 non-null    float64
 4   Taux de chômage                                                                    130 non-null    float64
 5   Taux d'inflation                                                                   130 non-null    float64

In [229]:
colonnes_avant = len(df_final.columns)
colonnes_manquantes = df_final.columns[df_final.isnull().mean() > 0.1]

print(colonnes_manquantes)


df_final = df_final.dropna(axis=1, thresh=int(len(df_final) * 0.9))

colonnes_apres = len(df_final.columns)
print(f"Le DataFrame a été nettoyé de {colonnes_avant - colonnes_apres} colonnes.")

Index(['Area_1', 'PIB par habitant', 'Population totale_2',
       'Taux d'urbanisation', 'Taux de chômage', 'Taux d'inflation',
       'Part des énergies renouvelables', 'Taux de mortalité infantile',
       'Taux d'électrification', 'Area_translated_1', 'Area_2',
       'Production_Viande de Volailles',
       'Importations - Quantité_Viande de Volailles',
       'Variation de stock_Viande de Volailles',
       'Disponibilité intérieure_Viande de Volailles',
       'Résidus_Viande de Volailles', 'Nourriture_Viande de Volailles',
       'Disponibilité alimentaire en quantité (kg/personne/an)_Viande de Volailles',
       'Disponibilité alimentaire (Kcal/personne/jour)_Viande de Volailles',
       'Disponibilité de protéines en quantité (g/personne/jour)_Viande de Volailles',
       'Disponibilité de matière grasse en quantité (g/personne/jour)_Viande de Volailles',
       'Exportations - Quantité_Viande de Volailles', 'Area_translated_2',
       'Area_3', 'Population totale_3', 'Area_t

<div style="border: 1px solid RGB(51,165,182);" >
<h3 style="margin: auto; padding: 20px; color: RGB(51,165,182); ">3.3 - Remplir les NaN</h3>
</div>

In [ ]:
from sklearn.impute import KNNImputer

# Initialisation de l'imputeur KNN avec k voisins
imputer = KNNImputer(n_neighbors=5)

# Imputation sur les données numériques
df_final_numeric = df_final.select_dtypes(include=[float])

# Appliquer KNN Imputer sur les colonnes numériques uniquement
df_final_imputed = pd.DataFrame(imputer.fit_transform(df_final_numeric), columns=df_final_numeric.columns)

# Réinjecter les valeurs imputées dans le DataFrame original
df_final.update(df_final_imputed)


In [ ]:
df_final.info()

<div style="background-color: RGB(51,165,182);" >
<h2 style="margin: auto; padding: 20px; color:#fff; ">4 - Export des données</h2>
</div>

In [81]:
# Exporter le DataFrame final en CSV
df_final.to_csv('df_final.csv', index=False)

In [ ]:
jupyter nbconvert --to markdown --stdout notebook.ipynb > output_notebook.txt